In [1]:
import numpy as np
import panda as pd


def raster(spike_dict,unitidx,stim_dict,eventidx,event_times,rasterfs):
    # spike_dict : spikes of a hole recording
    # unitidx : unit's spikes we want to look at
    # stim_dict : stimuli, used for the names of the events
    # eventidx : idx of event we want to look at
    # rasterfs : spike frequency
    import numpy as np
    
    stimevents=list(stim_dict.keys())
    cellids=list(spike_dict.keys())
    print('hey')
    event_name=stimevents[eventidx]
    cellid=cellids[unitidx]
    
    binlen=1.0/rasterfs
    h=np.array([])
    ff = (event_times['name']==event_name)
    ## pull out each epoch from the spike times, generate a raster of spike rate
    for i,d in event_times.loc[ff].iterrows():
        #print("{0}-{1}".format(d['start'],d['end']))
        edges=np.arange(d['start'],d['end']+binlen,binlen)
        th,e=np.histogram(spike_dict[cellid],edges)

        #print("{0}-{1}: {2}".format(edges[0],edges[1],sum((spike_dict[cellid]>edges[0]) & (spike_dict[cellid]<edges[1]))))
        th=np.reshape(th,[1,-1])
        if h.size==0:
            # lazy hack: intialize the raster matrix without knowing how many bins it will require
            h=th
        else:
            # concatenate this repetition, making sure binned length matches
            if th.shape[1]<h.shape[1]:
                h=np.concatenate((h,np.zeros([1,h.shape[1]])),axis=0)
                h[-1,:]=np.nan
                h[-1,:th.shape[1]]=th
            else:
                h=np.concatenate((h,th[:,:h.shape[1]]),axis=0)
    return h

def getTrainTestTimes(event_times,trainNb,testNb):
    # event_times : timings of events 
    # trainNb : number of stimuli presented for the trains
    # testNb : number of stimuli presented for the tests
    
    wavEvents = event_times[event_times['name'].str.contains('.wav')]
    occurences =  wavEvents['name'].value_counts(sort=True)

    Train_names = list(occurences[occurences==trainNb].index)
    Test_names = list(occurences[occurences==testNb].index)


    Train_times = pd.DataFrame(columns={'name','start','end'})
    Train_times = Train_times[['name','start','end']] #Order the columns
    Test_times = Train_times.copy()

    #Get stimuli onset and offset times for trains
    trial_indexs = event_times['name'][event_times['name']=='TRIAL'].index
    idx1 = 0; idx2 = 0;
    for trial_idx in trial_indexs:
        name = event_times.iloc[trial_idx+1]['name']
        if name in Train_names :
            Train_times.at[idx1,'name'] = name
            Train_times.at[idx1,'start'] = event_times.iloc[trial_idx+3]['end']
            Train_times.at[idx1,'end'] = event_times.iloc[trial_idx+4]['start']
            idx1 +=1
        elif name in Test_names :
            Test_times.at[idx2,'name'] = name
            Test_times.at[idx2,'start'] = event_times.iloc[trial_idx+3]['end']
            Test_times.at[idx2,'end'] = event_times.iloc[trial_idx+4]['start']
            idx2 +=1
        else : 
            raise ValueError('Neither a Test nor a Train stimuli name')

    Train_times = Train_times.sort_values('name')
    Test_times = Test_times.sort_values('name')

    return Train_times,Test_times

[NbConvertApp] Converting notebook run_fun.ipynb to script
[NbConvertApp] Writing 793 bytes to run_fun.py
